In [1]:
import gpkit
from gpkit import Model, Variable
from gpkit.constraints.tight import Tight
from gpkit.constraints.bounded import Bounded
from gpkit import ureg

from BasicRocketComponents import Motor, Tube

​

In [2]:

class Rocket(Model):
    
    def setup(self):
        
        constraints = []
        components = []
        
        m = self.m = Variable("m","kg","Mass of Rocket")
        
        #define all components
        
        #nc   = self.nc   = NoseCone()
        body = self.body = Tube()
        #fins = self.fins = Fins()
        av      = self.av      = Avionics()
        payload = self.payload = Payload()
        recovery  = self.recovery  = Recovery()
        mainMotor = self.mainMotor = MainMotor()
        boosters  = self.boosters  = Booster()
        
        #components += [nc]
        components += [body]
        #components += [fins]
        components += [av]
        components += [payload]
        components += [recovery]
        components += [mainMotor]
        components += [boosters]
        
        #propellant mass fraction needs to be 17% of total vehicle
        constraints += [mainMotor.motor.I_t >= 22000*ureg.N*ureg.s]
        
        #define mass of airframe based on dry mass
        constraints += [body.m >= 10*ureg.kg]
        
        #require boosters to clear the launch tower
        
        constraints += self.launch_rod_clearance()
        
        constraints +=  [Tight([self.m >= sum(comp.m for comp in components)])]
        return [components, constraints]
    
    
    def launch_rod_clearance(self):
        
        constraints = []
        
        # first performed by assuming the mass isnt changed
        #height of launch rod
        h_LR = self.h_LR = Variable("h_{LR}",5.2, "m", "Launch rod height")

        
        #height reached by boosters alone
        h = Variable("h","m","Height reached by boosters")
        
        t = Variable("t", "s", "Time of launch rod clearance")
        
        #velocity at launch rod clearance time
        v = Variable("v_{LR}", "m/s", "Velocity at launch rod clearance")
        
        g = Variable("g", 9.81,"m/s^2", "Acceleration due to gravity")
        
        constraints += [v<=(self.boosters.F_avg/self.m - g)*t]
        constraints += [h_LR>=0.5*(self.boosters.F_avg/self.m - g)*t**2]
        
        #velocity requirement
        v_minLR = Variable("v_{LR, min}", 30.5, "m/s", "Minimum velocity off the launch rail")
        constraints += [v>= v_minLR]
        
        
        constraints += [t <= self.boosters.motor.t_burn]
        #needs to return a list of constraints
        
        return constraints
        
        

In [3]:
class Avionics(Model):
    
    def setup(self):
        
        constraints = []
        
        m = self.m = Variable("m", "kg", "Mass of Avionics")
        
        constraints += [m >= 1*ureg.kg]
        
        return [constraints]

In [4]:
class Payload(Model):
    
    def setup(self):
        
        constraints = []
        m = self.m = Variable("m", "kg","Mass of payload")
        
        constraints += [m >= 5*ureg.kg]
        
        return [constraints]
        
    

In [5]:
class Recovery(Model):
    
    def setup(self):
        
        constraints = []
        m = self.m = Variable("m", "kg", "Mass of Recovery System")
        
        constraints += [m >= 7*ureg.kg]
        
        return [constraints]

In [6]:
class MainMotor(Model):
    
    def setup(self):
        
        constraints = []
        components = self.components = []
        m = self.m = Variable("m", "kg", "Mass of Main Motor")
        
        motor = self.motor = Motor()
        components += [motor]
        
        constraints += [motor.m_dry >= 0.7*self.m] #based on most rockets on thrustcurve.org
        constraints += [Tight([motor.F_avg >= 350*ureg.N])]
        constraints += [Tight([motor.t_burn >= 20*ureg.s])]
        
        constraints += [Tight([motor.c <= 2200*ureg.m/ureg.s])] #based on most rockets on thrustcurve.org
                                    
        constraints +=  Tight([self.m >= sum(comp.m for comp in components)])
        
        return [components, constraints]
    
class Booster(Model):
    
    def setup(self):
        
        constraints = []
        components = self.components = []
        m = self.m = Variable("m", "kg", "Mass of all boosters")
        
        
        num = Variable("num", 4, "", "Number of boosters")
        
        #motor referes to a single motor
        motor = self.motor = Motor()
        components += [motor]
        
        #generic bounds, computing the bounds based on the vehicle mass is done in the Rocket() class. 
        constraints += [motor.m_dry >= 0.6*motor.m]
        constraints += [motor.F_avg >= 10*ureg.N]
        constraints += [motor.F_avg <= 4000*ureg.N]
        constraints += [motor.t_burn >= 0.5*ureg.s]
        
        constraints += [Tight([motor.c <= 2200*ureg.m/ureg.s])]
        
        
        #constraint the total impulse by the burn time
        const1 = Variable("K1",2000,"N", "constant for limiting booster perf")
        constraints += [motor.I_t <= const1*motor.t_burn]
        const2 = Variable("K2",7,"s", "constant for limiting booster perf")
        constraints += [motor.I_t <= const2*motor.F_avg]
        const3 = Variable("K3",0.5,"s", "constant for limiting booster perf")
        constraints += [motor.I_t >= const3*motor.F_avg]
        
        
        #get average thrust of all the boosters
        F_avg = self.F_avg  = Variable("F_{avg}", "N", "Average force produced by all the boosters together")
        constraints += [F_avg <= num*motor.F_avg]
        
        
        #in this case, we define the mass constraint explicity
        constraints += [Tight([self.m >= num*motor.m])]
        
        return [components, constraints]

In [7]:
with gpkit.SignomialsEnabled():
    r = Rocket()

In [8]:
r

[[[],
  [[gpkit.PosynomialInequality(m_Rocket/Avionics >= 1)]],
  [[gpkit.PosynomialInequality(m_Rocket/Payload >= 5)]],
  [[gpkit.PosynomialInequality(m_Rocket/Recovery >= 7)]],
  [[[gpkit.PosynomialInequality(m_Rocket/MainMotor/Motor >= m_{dry}_Rocket/MainMotor/Motor + m_{prop}_Rocket/MainMotor/Motor),
     [gpkit.PosynomialInequality(T_{avg}_Rocket/MainMotor/Motor <= c_Rocket/MainMotor/Motor*m_{prop}_Rocket/MainMotor/Motor*t_{burn}_Rocket/MainMotor/Motor**-1)],
     [gpkit.PosynomialInequality(T_{avg}_Rocket/MainMotor/Motor >= c_Rocket/MainMotor/Motor*m_{prop}_Rocket/MainMotor/Motor*t_{burn}_Rocket/MainMotor/Motor**-1)],
     [gpkit.PosynomialInequality(I_t_Rocket/MainMotor/Motor <= c_Rocket/MainMotor/Motor*m_{prop}_Rocket/MainMotor/Motor)],
     [gpkit.PosynomialInequality(I_t_Rocket/MainMotor/Motor >= c_Rocket/MainMotor/Motor*m_{prop}_Rocket/MainMotor/Motor)]]],
   [gpkit.PosynomialInequality(m_{dry}_Rocket/MainMotor/Motor >= 0.7*m_Rocket/MainMotor),
    [gpkit.PosynomialInequalit

In [9]:
opt = Model(r.m, Bounded(r))

In [10]:
sol = opt.localsolve()

Beginning signomial solve.
Solving took 5 GP solves and 1.89 seconds.


In [11]:
print(sol.summary())


Cost
----
 59.7 [kg]

Warnings
Unexpectedly Loose Constraints
------------------------------
               | Rocket/MainMotor
   35.49 >= 20 : t_{burn}_/Motor >= 20
  619.9 >= 350 : T_{avg}_/Motor >= 350

               | Rocket/Booster
  2200 <= 2200 : c_/Motor <= 2.2e+03
3.367 >= 3.367 : m_ >= m_/Motor*num_

Free Variables
--------------
         | Rocket
       m : 59.7     [kg]  Mass of Rocket
       t : 0.341    [s]   Time of launch rod clearance
  v_{LR} : 30.5     [m/s] Velocity at launch rod clearance

         | Rocket/Avionics
       m : 1        [kg]  Mass of Avionics

         | Rocket/Booster
 F_{avg} : 5926     [N]   Average force produced by all the boosters together
       m : 3.367    [kg]  Mass of all boosters

         | Rocket/Booster/Motor
     I_t : 740.7    [N·s] Total impulse of motor
 T_{avg} : 1481     [N]   Average force of motor
       c : 2200     [m/s] Exhaust speed of motor
       m : 0.8417   [kg]  Total mass of motor
 m_{dry} : 0.505    [kg]  Dry Mass

In [12]:
#estimate drag force at launch rod clearance
0.5*1.225*30**2*0.5*3.14*(0.15/2)**2

4.8682265625